# Hudson Valley Gentrification and Changes in Farming

## Notebook 2: USDA Agriculture Census data

In [1]:
import pandas as pd
import requests
import os

In [2]:
api_key_nass = os.getenv("QUICK_STATS_API_KEY")

In [54]:
zipcodes = ['12577',
 '10953',
 '10930',
 '12158',
 '10990',
 '12054',
 '10950',
 '12143',
 '12529',
 '12586',
 '12589',
 '10980',
 '10509',
 '12545',
 '12543',
 '12582',
 '12729',
 '10933',
 '12518',
 '06757',
 '06068',
 '06784',
 '06069',
 '06039',
 '12123',
 '12169',
 '12427',
 '12015',
 '12423',
 '12444',
 '01254',
 '01237',
 '01258',
 '18458',
 '18405',
 '18461',
 '18417',
 '18415',
 '18435',
 '18336',
 '18340',
 '01267',
 '01230',
 '01201',
 '01266',
 '07422',
 '07827',
 '07456',
 '07418',
 '07462',
 '07461',
 '07421',
 '12590',
 '12062',
 '12168',
 '12018',
 '12468',
 '12482',
 '12042',
 '12569',
 '12424',
 '12473',
 '12553',
 '10969',
 '12460',
 '10975',
 '12538',
 '12138',
 '12153',
 '12061',
 '12603',
 '12413',
 '12051',
 '12480',
 '12439',
 '12485',
 '12463',
 '12581',
 '12734',
 '12448',
 '12787',
 '12428',
 '12726',
 '12762',
 '12751',
 '12421',
 '12513',
 '12173',
 '12195',
 '12498',
 '12490',
 '12457',
 '12477',
 '12791',
 '12741',
 '12776',
 '12733',
 '12481',
 '12494',
 '12701',
 '12406',
 '12764',
 '12487',
 '12429',
 '12471',
 '12743',
 '12453',
 '12765',
 '12076',
 '12759',
 '12449',
 '12433',
 '12724',
 '12483',
 '12465',
 '12446',
 '12752',
 '12486',
 '12472',
 '12017',
 '12491',
 '12738',
 '12475',
 '12455',
 '12722',
 '12784',
 '12737',
 '12484',
 '12489',
 '12528',
 '12561',
 '12417',
 '13856',
 '12106',
 '12125',
 '12037',
 '12530',
 '12174',
 '12514',
 '13755',
 '12172',
 '12516',
 '12517',
 '12502',
 '12571',
 '12534',
 '12522',
 '12580',
 '10940',
 '12771',
 '10579',
 '10941',
 '10541',
 '12526',
 '10516',
 '10963',
 '10915',
 '10924',
 '12566',
 '10918',
 '12520',
 '10917',
 '12041',
 '12067',
 '12087',
 '12193',
 '10914',
 '12147',
 '10925',
 '10535',
 '12540',
 '12063',
 '12496',
 '12521',
 '10932',
 '10916',
 '10921',
 '12186',
 '10996',
 '12059',
 '10986',
 '10512',
 '12414',
 '12442',
 '12451',
 '12176',
 '12492',
 '12454',
 '12585',
 '12507',
 '12504',
 '12604',
 '12512',
 '12758',
 '12740',
 '12458',
 '12409',
 '12723',
 '12789',
 '12790',
 '12410',
 '12736',
 '12767',
 '12435',
 '12404',
 '12464',
 '12719',
 '12792',
 '12745',
 '12777',
 '12411',
 '12781',
 '12750',
 '10911',
 '12721',
 '12785',
 '10988',
 '10992',
 '12077',
 '10922',
 '12161',
 '12023',
 '10910',
 '12570',
 '12578',
 '12033',
 '12156',
 '12788',
 '12783',
 '12748',
 '12760',
 '12766',
 '12401',
 '12441',
 '12720',
 '12768',
 '12725',
 '12754',
 '12525',
 '12419',
 '12515',
 '12786',
 '13731',
 '12565',
 '12132',
 '12436',
 '12431',
 '12058',
 '12508',
 '12192',
 '12418',
 '12124',
 '12470',
 '12452',
 '12450',
 '12416',
 '12747',
 '12432',
 '12779',
 '12763',
 '12443',
 '12775',
 '12567',
 '12503',
 '12563',
 '12501',
 '12542',
 '12549',
 '10524',
 '10588',
 '10985',
 '12060',
 '12115',
 '12544',
 '12594',
 '12572',
 '12546',
 '12769',
 '12466',
 '12778',
 '12742',
 '12547',
 '12548',
 '13756',
 '13774',
 '12029',
 '12165',
 '12130',
 '13783',
 '12469',
 '10987',
 '12144',
 '12524',
 '12024',
 '12022',
 '12527',
 '12601',
 '12550',
 '12592',
 '10919',
 '12575',
 '12746',
 '10958',
 '10973',
 '12574',
 '12495',
 '12412',
 '12461',
 '12456',
 '12420',
 '12732',
 '12770',
 '12440',
 '12749',
 '12493',
 '12136',
 '12075',
 '12046',
 '12045',
 '12007',
 '12083',
 '10979',
 '12184',
 '12430',
 '12405',
 '12422',
 '12583',
 '12202',
 '12523',
 '12533',
 '12780',
 '10974',
 '10998',
 '10926',
 '10928',
 '12564',
 '12531',
 '12120']

## Download Economics and Demographics sectors from Agriculture Census and identify relevant variables

In [ ]:
source_desc = "CENSUS"
sector_desc = ["Economics", "Demographics"]
group_desc = ['EXPENSES','FARMS & LAND & ASSETS','INCOME','PRODUCERS']
# These are the columns, in order, that (within a given sector) sequentially identify variables at any given level
group_cols = ["group_desc","commodity_desc", "class_desc", "prodn_practice_desc", "statisticcat_desc", "unit_desc","domain_desc","domaincat_desc"]

### Check whether there are fewer than 50,000 results at each specified year/level/sector

In [4]:
num_results = []
for year in ["2017","2022"]:
    for level in ["ZIP CODE", 'COUNTY']:
        for sector in sector_desc:
            r = requests.get(url=f'https://quickstats.nass.usda.gov/api/get_counts/?key={api_key_nass}', params={"sector_desc": sector, 
                                                                                                            "year": year, 
                                                                                                            "state_fips_code": "36", 
                                                                                                            "agg_level_desc": level}).json()
            num_results.append((year, level, sector, r['count']))

In [5]:
num_results

[('2017', 'ZIP CODE', 'Economics', 24414),
 ('2017', 'ZIP CODE', 'Demographics', 19578),
 ('2017', 'COUNTY', 'Economics', 24958),
 ('2017', 'COUNTY', 'Demographics', 17316),
 ('2022', 'ZIP CODE', 'Economics', 23609),
 ('2022', 'ZIP CODE', 'Demographics', 18113),
 ('2022', 'COUNTY', 'Economics', 24697),
 ('2022', 'COUNTY', 'Demographics', 13755)]

### Perform API calls for specified year/aggregation/sector; store results and transform into dataframes

In [16]:
ag_census_results = {}
for year in ["2017","2022"]:
    ag_census_results[year] = {}
    for level in ["zip code", 'county']:
        ag_census_results[year][level] = {}
        for sector in sector_desc:
            r = requests.get(url=f"https://quickstats.nass.usda.gov/api/api_GET/?key={api_key_nass}", 
                            params={"sector_desc": sector, 
                                    "year": year, 
                                    "state_fips_code": "36", 
                                    "agg_level_desc": level}).json()
            ag_census_results[year][level][sector] = r

In [17]:
for year in ["2017","2022"]:
    for level in ["zip code", 'county']:
        for sector in sector_desc:
            ag_census_results[year][level][sector]['df'] = pd.DataFrame.from_dict(ag_census_results[year][level][sector]['data'])

In [18]:
ag_census_results['2017']['county']['Economics']['df'].group_desc.unique()

array(['ENERGY', 'EXPENSES', 'FARMS & LAND & ASSETS', 'INCOME'],
      dtype=object)

In [24]:
for col in ag_census_results['2017']['county']['Economics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1).columns:
    print(col)
    print(ag_census_results['2017']['county']['Economics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1)[col].unique())

group_desc
['ENERGY' 'EXPENSES' 'FARMS & LAND & ASSETS' 'INCOME']
commodity_desc
['ENERGY' 'AG SERVICES' 'ANIMAL TOTALS' 'CHEMICAL TOTALS' 'DEPRECIATION'
 'EXPENSE TOTALS' 'FEED' 'FERTILIZER TOTALS' 'FUELS' 'INTEREST' 'LABOR'
 'RENT' 'SEEDS' 'SEEDS & PLANTS TOTALS' 'SUPPLIES & REPAIRS' 'TAXES'
 'AG LAND' 'FARM OPERATIONS' 'GOVT PROGRAMS' 'LAND AREA'
 'MACHINERY TOTALS' 'MACHINERY, OTHER' 'PRACTICES' 'SELF PROPELLED'
 'TRACTORS' 'TRUCKS' 'CCC LOANS' 'COMMODITY TOTALS' 'INCOME, FARM-RELATED'
 'INCOME, NET CASH FARM']
class_desc
['RENEWABLE, HARVEST BIOMASS FOR PRODUCTION'
 'CUSTOM SERVICES FOR LIVESTOCK, INCL MEDICAL SUPPLIES & VETERINARY'
 'CUSTOMWORK' 'MACHINERY RENTAL' 'OTHER' 'UTILITIES' '(EXCL BREEDING)'
 'ALL CLASSES' 'BREEDING' 'OPERATING' 'OPERATING, PAID BY LANDLORD'
 'INCL LIME & SOIL CONDITIONERS' 'INCL LUBRICANTS' 'NON-REAL ESTATE'
 'REAL ESTATE' 'CONTRACT' 'HIRED' 'HIRED, GE 150 DAYS'
 'HIRED, LT 150 DAYS' 'MIGRANT' 'UNPAID' 'CASH, CROPLAND'
 'CASH, LAND & BUILDINGS' 'CASH, 

In [26]:
for col in ag_census_results['2017']['county']['Demographics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1).columns:
    print(col)
    print(ag_census_results['2017']['county']['Demographics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1)[col].unique())

group_desc
['ANIMAL TOTALS' 'AQUACULTURE' 'CROP TOTALS' 'DAIRY' 'EXPENSES'
 'FARMS & LAND & ASSETS' 'FIELD CROPS' 'FRUIT & TREE NUTS' 'HORTICULTURE'
 'INCOME' 'LIVESTOCK' 'POULTRY' 'PRODUCERS' 'SPECIALTY' 'VEGETABLES']
commodity_desc
['ANIMAL TOTALS' 'AQUACULTURE TOTALS' 'CROP TOTALS' 'MILK'
 'EXPENSE TOTALS' 'LABOR' 'AG LAND' 'FARM OPERATIONS' 'INTERNET'
 'PRACTICES' 'BARLEY' 'CORN' 'FIELD CROPS, OTHER' 'GRAIN' 'HAY & HAYLAGE'
 'OATS' 'SOYBEANS' 'WHEAT' 'APPLES' 'FRUIT & TREE NUT TOTALS' 'GRAPES'
 'PEACHES' 'CUT CHRISTMAS TREES'
 'CUT CHRISTMAS TREES & SHORT TERM WOODY TREES' 'HORTICULTURE TOTALS'
 'NURSERY TOTALS' 'SOD' 'COMMODITY TOTALS' 'GOVT PROGRAMS'
 'INCOME, FARM-RELATED' 'INCOME, NET CASH FARM' 'CATTLE' 'GOATS' 'HOGS'
 'SHEEP' 'SHEEP & GOATS TOTALS' 'CHICKENS' 'POULTRY TOTALS' 'TURKEYS'
 'PRODUCERS' 'PRODUCERS, PRINCIPAL' 'EQUINE' 'SPECIALTY ANIMAL TOTALS'
 'POTATOES' 'SQUASH' 'SWEET CORN' 'VEGETABLE TOTALS']
class_desc
['INCL PRODUCTS' 'ALL CLASSES' 'OPERATING' 'HIRED'
 '(EXC

In [ ]:
# save raw data to CSVs
for year in ["2017","2022"]:
    for level in ["zip code", 'county']:
        for sector in sector_desc:
            ag_census_results[year][level][sector]['df'].to_csv(f"data/ag_census_{"".join(level.split(" "))}_{year}_{sector}.csv", index=False, encoding='utf-8')

In [ ]:
# save variables to CSVs
for year in ["2017","2022"]:
    for level in ["zip code", 'county']:
        for sector in sector_desc:
            df = ag_census_results[year][level][sector]['df'].groupby(group_cols).size().reset_index().drop(0, axis=1)
            df.to_csv(f"data/ag_census_vars/ag_census_{"".join(level.split(" "))}_{year}_{sector}_variables.csv", index=False, encoding='utf-8')

In [34]:
econ2017_zipcode_df = ag_census_results['2017']['zip code']['Economics']['df']
econ2022_zipcode_df = ag_census_results['2022']['zip code']['Economics']['df']
econ2017_county_df = ag_census_results['2017']['county']['Economics']['df']
econ2022_county_df = ag_census_results['2022']['county']['Economics']['df']

demog2017_zipcode_df = ag_census_results['2017']['zip code']['Demographics']['df']
demog2022_zipcode_df = ag_census_results['2022']['zip code']['Demographics']['df']
demog2017_county_df = ag_census_results['2017']['county']['Demographics']['df']
demog2022_county_df = ag_census_results['2022']['county']['Demographics']['df']

In [ ]:
econ2017_zipcode_df_subset = econ2017_zipcode_df.query("group_desc in ['FARMS & LAND & ASSETS','INCOME'] and commodity_desc in ['AG LAND','COMMODITY TOTALS'] and class_desc in ['CROPLAND, HARVESTED','ALL CLASSES'] and prodn_practice_desc != 'ORGANIC'").copy()
econ2022_zipcode_df_subset = econ2022_zipcode_df.query("group_desc in ['FARMS & LAND & ASSETS','INCOME'] and commodity_desc in ['AG LAND','COMMODITY TOTALS'] and class_desc in ['CROPLAND, HARVESTED','ALL CLASSES'] and prodn_practice_desc != 'ORGANIC'").copy()

In [50]:
demog2017_zipcode_df_subset = demog2017_zipcode_df.query("domain_desc == 'TENURE' or prodn_practice_desc == 'PRIMARY OCCUPATION, FARMING'").copy()
demog2022_zipcode_df_subset = demog2022_zipcode_df.query("domain_desc == 'TENURE' or prodn_practice_desc == 'PRIMARY OCCUPATION, FARMING'").copy()

In [57]:
econ2017_zipcode_df_core = econ2017_zipcode_df_subset.query("zip_5 in @zipcodes").copy()
econ2022_zipcode_df_core = econ2022_zipcode_df_subset.query("zip_5 in @zipcodes").copy()
demog2017_zipcode_df_core = demog2017_zipcode_df_subset.query("zip_5 in @zipcodes").copy()
demog2022_zipcode_df_core = demog2022_zipcode_df_subset.query("zip_5 in @zipcodes").copy()